In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve
import scikitplot as skplt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import time
import os
print(os.listdir("../input/6f970556-9-dataset_-_problem_2/Dataset - Problem 2"))
PATH = "../input/6f970556-9-dataset_-_problem_2/Dataset - Problem 2"
# Any results you write to the current directory are saved as output.

Binary-class data with 406709 rows and 54 variables (10 Numerical and 44 Categorical). The header file is provided whose first row mentions the variable names and second row mentions the type of those respective variables (Numerical, Categorical). The first column of the data is the unique identifier for each row. The last column mentions the class/label for each row.

In [ ]:
header = pd.read_csv(PATH+'/header.csv')
display (header.head())
print (header.shape)

In [ ]:
d =['V'+str(i) for i in range(1, 55)]
cols = ['key'] + d + ['L']
print (cols)

In [ ]:
train_data = pd.read_csv(PATH+'/train.csv', header=None, names=cols)
display (train_data.head())
print (train_data.shape)

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.iloc[:,11:-1].sum(axis=1).sum()/406709.0 ## two categorical variables one hotted into 44 categorical variables

In [ ]:
## find the two categorical varibles
for i in range(11,56):
    print (train_data.iloc[:,11:i].sum(axis=1).sum()/406709.0,i)
    
## 11-15 one categorical variable, 16-55 another one

In [ ]:
print (train_data.iloc[:, 11:15].values)
train_data['cat_1'] = train_data.iloc[:, 11:15].values.argmax(axis=1)
print (train_data.iloc[:, 11:15].values.argmax(axis=1))

In [ ]:
print (train_data.iloc[:, 15:55].values)
train_data['cat_2'] = train_data.iloc[:, 15:55].values.argmax(axis=1)
print (train_data.iloc[:, 15:55].values.argmax(axis=1))

In [ ]:
train_data['sum']=train_data.loc[:,'V1':'V10'].sum(axis=1)

In [ ]:
display(train_data.head())

In [ ]:
new_cols = list(train_data.iloc[:, 11:55].columns)
print (new_cols)
train_new_data = train_data.drop(new_cols, axis=1)
display(train_new_data.head())
print (train_new_data.shape)

In [ ]:
sns.countplot(x='L', data=train_data)
print (train_data.L.value_counts())

In [ ]:
sns.heatmap(train_data.corr())

In [ ]:
mm = StandardScaler()
categories_cols = []
numeric_cols = []

for c in train_data.columns:
    if header[c].values == 'C':
        categories_cols.append(c)
    if header[c].values == 'N':
        numeric_cols.append(c)
train_num = train_data[numeric_cols]
train_cat = train_data[categories_cols]

print (train_num.shape, train_cat.shape, train_data.shape)

In [ ]:
#optional
train_num = mm.fit_transform(train_num)
train_num = pd.DataFrame(train_num, columns=numeric_cols)
display (train_num.head())

In [ ]:
train_data.update(train_num)
display (train_data.head())
print (train_data.shape)

In [ ]:
#Using target enconding

import category_encoders as ce
target_encoding = ce.target_encoder.TargetEncoder(cols=['cat_2'])
data = train_new_data[['cat_2']]
display(data.head())
print (data.shape, labels.shape)
data_new = target_encoding.fit_transform(data, labels)

train_new = train_new_data
train_new.update(data_new)
display(train_new.head())

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=20)
# train_pca_data = pca.fit_transform(train_data)
# print (train_pca_data.shape)

In [ ]:
labels = train_new_data['L']
train_new_data = train_new_data.drop(['L'], axis=1)
print (train_new_data.shape, labels.shape)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_new_data, labels, test_size=0.2, stratify=labels)
print ('Training Shape', train_x.shape, train_y.shape)
print ('Validation Shape', val_x.shape, val_y.shape)

# Model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifiers = [
    LogisticRegression(C=0.000000001, max_iter=400),
    KNeighborsClassifier(2),
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=500),
    ExtraTreesClassifier(n_estimators=500),
    AdaBoostClassifier(n_estimators=50)]

# Test Data Preprocessing

In [ ]:
test_data = pd.read_csv(PATH+'/test.csv', header=None, names=cols)
test_data = test_data.drop(['L'], axis=1)
display (test_data.head())
print (test_data.shape)

In [ ]:
test_data.info()

In [ ]:
test_data['cat_1'] = test_data.iloc[:, 11:15].values.argmax(axis=1)
test_data['cat_2'] = test_data.iloc[:, 15:55].values.argmax(axis=1)
test_data['sum'] = test_data.loc[:,'V1':'V10'].sum(axis=1)

test_new_data = test_data.drop(new_cols, axis=1)
display(test_new_data.head())
print (test_new_data.shape)

In [ ]:
test_num = test_data[numeric_cols]

test_num = mm.fit_transform(test_num)
test_num = pd.DataFrame(test_num, columns=numeric_cols)
display (test_num.head())

In [ ]:
test_data.update(test_num)
display (test_data.head())
print (test_data.shape)

In [ ]:
# test_pca_data = pca.transform(test_data)
# print (test_pca_data.shape)

In [ ]:
Accuracy=[]
Model=[]
for clf in classifiers:
    start = time.time()
    clf.fit(train_x, train_y)
    pred = clf.predict(val_x)
    auc = roc_auc_score(val_y, pred)
    Accuracy.append(auc)
    Model.append(clf.__class__.__name__)
    print('Auc score of '+ clf.__class__.__name__+' is '+ str(auc))
    print ('Time taken for training', time.time()-start, 'sec')
    test_pred = clf.predict_proba(test_new_data)
    pred = test_pred[:, 1]
    sub = pd.DataFrame()
    sub['key'] = test_data['key']
    sub['score'] = pred
    sub.to_csv('submit_'+clf.__class__.__name__+'.csv', index=False)

In [ ]:
# Index = [1,2,3,4,5,6,7]
# plt.bar(Index,Accuracy)
# plt.xticks(Index, Model, rotation=45)
# plt.ylabel('Accuracy')
# plt.xlabel('Model')
# plt.title('Auc scores of Models')

# Stacking Models

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score

clf1 = RandomForestClassifier(n_estimators=500, max_features=50)
clf2 = ExtraTreesClassifier(n_estimators=500, max_features=50)
lr = LogisticRegression()
clf3 = BaggingClassifier(n_estimators=500, random_state=42, max_features=50)
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

for clf, label in zip([clf1, clf2, clf3, sclf], 
                      [ 'Random Forest', 
                       'Extra Trees',
                       'Bagging',
                       'StackingClassifier']):

    clf.fit(train_x, train_y)
    pred = clf.predict(val_x)
    auc = roc_auc_score(val_y, pred)
    test_pred = clf.predict_proba(test_data)
    pred = test_pred[:, 1]
    sub = pd.DataFrame()
    sub['key'] = test_data['key']
    sub['score'] = pred
    sub.to_csv('submit_'+label+'.csv', index=False)
    print("AUC:", (auc))

# XGB Boost

In [ ]:
import xgboost as xgb

# # Parameter Tuning
# model = xgb.XGBClassifier()
# param_dist = {"max_depth": [10,30,50],
#               "min_child_weight" : [1,3,6],
#               "n_estimators": [200],
#               "learning_rate": [0.05, 0.1,0.16]}
# grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
#                                    scoring='accuracy', verbose=10, n_jobs=-1)
# grid_search.fit(train_x, train_y)
# print (grid_search.best_estimator_)
#max_depth=50, min_child_weight=1,  n_estimators=200,\
#                          n_jobs=-1 , verbose=1,learning_rate=0.16
model = xgb.XGBClassifier(n_estimators=500, max_depth=50, booster='dart')
model.fit(train_x, train_y)
pred = model.predict(val_x)
auc = roc_auc_score(val_y, pred)

test_pred = model.predict_proba(test_data)
pred = test_pred[:, 1]
sub = pd.DataFrame()
sub['key'] = test_data['key']
sub['score'] = pred
sub.to_csv('submit_xgb.csv', index=False)
print("AUC:", (auc))

# LightGBM

In [ ]:
import lightgbm as lgb

# #parameter tuning
model = lgb.LGBMClassifier(n_estimators=500, boosting_type='dart', silent=False, max_depth=20, num_leaves=1000)
# param_dist = {"max_depth": [25,50, 75],
#               "learning_rate" : [0.01,0.05,0.1],
#               "num_leaves": [300,900,1200],
#               "n_estimators": [200]
#              }
# grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="accuracy", verbose=5)
# grid_search.fit(train_x, train_y)
# print (grid_search.best_estimator_)

# d_train = lgb.Dataset(train_x, label=train_y)
# params = {"max_depth": 50, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 300}

# # #With Catgeorical Features
model.fit(train_x, train_y, categorical_feature = list(categories_cols))
pred = model.predict(val_x)
auc = roc_auc_score(val_y, pred)

test_pred = model.predict_proba(test_data)
pred = test_pred[:, 1]
sub = pd.DataFrame()
sub['key'] = test_data['key']
sub['score'] = pred
sub.to_csv('submit_lgb.csv', index=False)
print("AUC:", (auc))

In [ ]:
import xgboost as xgb

## gradient boosting
def run(train,lables,test,params):
    dtrain=xgb.DMatrix(train,label=lables)
    dtest=xgb.DMatrix(test)
    bst=xgb.train(dtrain=dtrain,params=params,num_boost_round=700)
    preds=bst.predict(dtest)
    return bst,preds

params={
    'objective':'binary:logistic',
    'max_depth':27,
    'eta':0.08,
    'subsample':0.9,
#     'eval_metric': 'auc'
}

In [ ]:
start = time.time()
bst,probs=run(train_x,train_y,val_x,params)
print (roc_auc_score(val_y,probs))
print ('Time taken for training', time.time()-start, 'sec')
# 0.9984964681315754 no target encoding
# 0.9985171552493545 both cat_1 and cat_2
# 0.9984994316004411 only cat_1 